In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17725123178752385632, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10912176626631037263
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 16
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)
epochs = 50

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [7]:
from keras.layers import Conv2D
from keras.models import Sequential


def SmallCNN(img_size):
    model = Sequential()
    model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(img_size, img_size, 3)))
    model.add(Conv2D(32, 3, activation='relu', padding='same'))
    model.add(Conv2D(1, 5, activation='sigmoid', padding='same'))

    return model

In [8]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [9]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [10]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 1632
predict_size_validation: 544
predict_size_test: 2


In [11]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [12]:
model = SmallCNN(img_size = 256)
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 32)      4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 1)       801       
Total params: 5,889
Trainable params: 5,889
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    verbose = 2,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/50
 - 359s - loss: -1.2475e-01 - dice_coef: 0.1247 - acc: 0.4079 - mean_squared_error: 0.3587 - Jaccard_index: 0.0667 - Specificity: 0.9731 - Sensitivity: 1.4842 - precision: 0.0758 - recall: 0.7782 - f1score: 0.1375 - val_loss: -1.5984e-01 - val_dice_coef: 0.1598 - val_acc: 0.4998 - val_mean_squared_error: 0.2883 - val_Jaccard_index: 0.0873 - val_Specificity: 0.9648 - val_Sensitivity: 1.3732 - val_precision: 0.0944 - val_recall: 0.6313 - val_f1score: 0.1632
Epoch 2/50
 - 356s - loss: -1.4226e-01 - dice_coef: 0.1423 - acc: 0.5214 - mean_squared_error: 0.2880 - Jaccard_index: 0.0768 - Specificity: 0.9710 - Sensitivity: 1.3572 - precision: 0.0839 - recall: 0.6409 - f1score: 0.1475 - val_loss: -1.6846e-01 - val_dice_coef: 0.1685 - val_acc: 0.5348 - val_mean_squared_error: 0.2782 - val_Jaccard_index: 0.0925 - val_Specificity: 0.9639 - val_Sensitivity: 1.3322 - val_precision: 0.1003 - val_recall: 0.6038 - val_f1score: 0.1708
Epoch 3/50
 - 355s - loss: -1.4669e-01 - dice_coef: 0.1467

Epoch 19/50
 - 356s - loss: -1.5754e-01 - dice_coef: 0.1575 - acc: 0.6029 - mean_squared_error: 0.2433 - Jaccard_index: 0.0859 - Specificity: 0.9680 - Sensitivity: 1.2576 - precision: 0.0968 - recall: 0.5448 - f1score: 0.1628 - val_loss: -1.9369e-01 - val_dice_coef: 0.1937 - val_acc: 0.6201 - val_mean_squared_error: 0.2225 - val_Jaccard_index: 0.1082 - val_Specificity: 0.9605 - val_Sensitivity: 1.2293 - val_precision: 0.1264 - val_recall: 0.5377 - val_f1score: 0.2016
Epoch 20/50
 - 356s - loss: -1.5844e-01 - dice_coef: 0.1584 - acc: 0.6059 - mean_squared_error: 0.2423 - Jaccard_index: 0.0865 - Specificity: 0.9677 - Sensitivity: 1.2553 - precision: 0.0977 - recall: 0.5431 - f1score: 0.1639 - val_loss: -1.9222e-01 - val_dice_coef: 0.1922 - val_acc: 0.6094 - val_mean_squared_error: 0.2344 - val_Jaccard_index: 0.1073 - val_Specificity: 0.9610 - val_Sensitivity: 1.2420 - val_precision: 0.1241 - val_recall: 0.5561 - val_f1score: 0.2004
Epoch 21/50
 - 356s - loss: -1.5742e-01 - dice_coef: 0.1

Epoch 37/50
 - 356s - loss: -1.7265e-01 - dice_coef: 0.1727 - acc: 0.6701 - mean_squared_error: 0.1771 - Jaccard_index: 0.0951 - Specificity: 0.9657 - Sensitivity: 1.1779 - precision: 0.1141 - recall: 0.4501 - f1score: 0.1789 - val_loss: -2.2209e-01 - val_dice_coef: 0.2221 - val_acc: 0.6995 - val_mean_squared_error: 0.1503 - val_Jaccard_index: 0.1267 - val_Specificity: 0.9574 - val_Sensitivity: 1.1365 - val_precision: 0.1634 - val_recall: 0.4367 - val_f1score: 0.2326
Epoch 38/50
 - 356s - loss: -1.7230e-01 - dice_coef: 0.1723 - acc: 0.6690 - mean_squared_error: 0.1790 - Jaccard_index: 0.0949 - Specificity: 0.9655 - Sensitivity: 1.1794 - precision: 0.1136 - recall: 0.4514 - f1score: 0.1785 - val_loss: -2.2295e-01 - val_dice_coef: 0.2230 - val_acc: 0.6787 - val_mean_squared_error: 0.1708 - val_Jaccard_index: 0.1270 - val_Specificity: 0.9589 - val_Sensitivity: 1.1634 - val_precision: 0.1591 - val_recall: 0.4950 - val_f1score: 0.2361
Epoch 39/50
 - 355s - loss: -1.7240e-01 - dice_coef: 0.1

In [ ]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

In [ ]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

In [ ]:
prediction.shape

In [ ]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

In [ ]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

In [ ]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()